# Test spike train distance EFeature

Compare spike trains obtained from protocol responses using spike train distance metrics.

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..
%pwd

In [ ]:
# BluePyOpt
import bluepyopt.ephys as ephys

# Our custom BluePyOpt modules
from optimize.bpop_cellmodels import StnFullModel, StnReducedModel

import optimize.bpop_protocols_stn as StnProtocols
from optimize.bpop_protocols_stn import BpopProtocolWrapper

import optimize.bpop_features_stn as StnFeatures
import optimize.bpop_parameters_stn as StnParameters

from optimize.bpop_analysis_stn import (
    run_proto_responses, plot_proto_responses, 
    save_proto_responses, load_proto_responses,
    plot_responses
)

# Physiology parameters
from evalmodel.cellpopdata import StnModel
from evalmodel.proto_common import StimProtocol

CLAMP_PLATEAU = StimProtocol.CLAMP_PLATEAU
CLAMP_REBOUND = StimProtocol.CLAMP_REBOUND
MIN_SYN_BURST = StimProtocol.MIN_SYN_BURST
SYN_BACKGROUND_HIGH = StimProtocol.SYN_BACKGROUND_HIGH

In [ ]:
# Adjust verbosity of loggers
import logging
silent_loggers = ['marasco', 'redops', 'folding', 'bluepyopt.ephys.parameters']
verbose_loggers = []
for logname in silent_loggers:
	logger = logging.getLogger(logname)
	if logger: logger.setLevel(logging.WARNING)

## Run full model

In [ ]:
# Make protocols
model_type = StnModel.Gillies2005

opt_stim_protocols = [SYN_BACKGROUND_HIGH]
proto_ids_wrappers = {stim_proto: BpopProtocolWrapper.make(stim_proto, model_type) for stim_proto in opt_stim_protocols}

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(proto_ids_wrappers.values())

# Distinguish between sets of parameters (used, frozen, free/optimised)
frozen_params = proto_params
free_params = []
used_params = frozen_params + free_params

# make cell model
full_model = StnFullModel(
                name		= 'StnGillies',
                mechs		= proto_mechs,
                params		= used_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

for proto_id in opt_stim_protocols:
    
    # Make protocol
    proto = proto_ids_wrappers[proto_id]

    # Compute protocol responses
    responses = proto.ephys_protocol.run(
                    cell_model		= full_model, 
                    param_values	= {},
                    sim				= nrnsim,
                    isolate			= True)

    # Plot responses
    plot_responses(responses)

## Features reduced full model

In [ ]:
# # Get protocol responses for full model
# PROTO_RESPONSES_FILE = "/home/luye/cloudstore_m/simdata/fullmodel/STN_Gillies2005_proto_responses.pkl" # backup is in filename.old.pkl
# full_responses = load_proto_responses(PROTO_RESPONSES_FILE)

# # Make protocols
# model_type = StnModel.Gillies_FoldMarasco
# opt_stim_protocols = [SYN_BACKGROUND_HIGH, CLAMP_PLATEAU, CLAMP_REBOUND, MIN_SYN_BURST]
# proto_ids_wrappers = {stim_proto: BpopProtocolWrapper.make(stim_proto, model_type) for stim_proto in opt_stim_protocols}
# all_proto_wrappers = proto_ids_wrappers.values()

# # Make EFEL feature objects
# stimprotos_feats = StnFeatures.make_opt_features(all_proto_wrappers) # dict {stim_protocol : (efeature, weight)}

# # Calculate target values from full model responses
# StnFeatures.calc_feature_targets(stimprotos_feats, full_responses)

## Run reduced model

In [ ]:
# Make protocols
model_type = StnModel.Gillies_FoldMarasco

opt_stim_protocols = [SYN_BACKGROUND_HIGH]
proto_ids_wrappers = {stim_proto: BpopProtocolWrapper.make(stim_proto, model_type) for stim_proto in opt_stim_protocols}

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(proto_ids_wrappers.values())

# Distinguish between sets of parameters (used, frozen, free/optimised)
frozen_params = proto_params
free_params = []
used_params = frozen_params + free_params

# Create reduced model and get parameters
red_model = StnReducedModel(
                name		= 'StnFolded',
                fold_method	= 'marasco',
                num_passes	= 7,
                mechs		= proto_mechs,
                params		= used_params)

default_params = {k : v*1.0 for k,v in StnParameters.default_params.items() if k in [p.name for p in used_params]}

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

for proto_id in opt_stim_protocols:
    
    # Get protocol wrapper
    proto = proto_ids_wrappers[proto_id]

    # Compute protocol responses
    responses = proto.ephys_protocol.run(
                    cell_model		= red_model, 
                    param_values	= {},
                    sim				= nrnsim,
                    isolate			= True) # allows us to query cell with h.allsec()

    # Plot responses
    # print("Full model responses:")
    # plot_responses(full_responses[proto_id.name])

    print("Reduced model responses:")
    plot_responses(responses)

    # Calculate response scores
#     featdict = stimprotos_feats[proto_id]
    
#     feat_names, feat_data = zip(*featdict.items())
#     efeats, weights = zip(*feat_data)

#     for efeat in efeats:
#         score = efeat.calculate_score(responses)
#         print('Score = <{}> for feature {}.{}'.format(score, stim_proto.name, efeat.name))